#Sistema di raccomandazione
Il sistema implementa un approccio di raccomandazione one-time, evitando la costruzione di matrici di similarità NxN. Applica un filtro hard sulla categoria e combina una similarità basata su ingredienti con una funzione deterministica di brand e segmento, pesate rispettivamente 0.8 e 0.2.



In [88]:
import pandas as pd
import numpy as np



1) Importo il df e creo un nuovo df con le colonne d'interesse



In [ ]:
df = pd.read_csv("brand_categoria.csv")
#df_ingredients= #inserire dataset ingredienti

df = df[['code', 'product_name','brand_name', 'brand_segment','macro_category']].copy()
df['code'] = df['code'].astype(str)
df['product_name'] = df['product_name'].astype(str)
df['brand_name'] = df['brand_name'].astype(str)
df['brand_segment'] = df['brand_segment'].astype(str)
df['macro_category'] = df['macro_category'].astype(str)
#inserire nel nuovo dataset le colonne rilevanti da df_ingredients

df.head(20)


,code,product_name,brand_name,brand_segment,macro_category
0,88582000939,2,1,mass_market,Other
1,0650240025396,zan zusi b b flash 30,1,mass_market,Other
2,5013965698897,1,1,mass_market,Other
3,3560070084074,deo men marine,1 de carrefour,mass_market,Deodorants
4,6134598000044,savon liquide main life,100 da,mass_market,Hygiene
5,5943044010046,gerocossen,12,mass_market,Other
6,20546472,cien,12,mass_market,Other
7,3700216252688,glycerine vegetale,123gelules,mass_market,Other
8,8859423200212,habino,13000,mass_market,Other
9,6130460000747,nan,150da,mass_market,Other


**2. Assegnazione pesi ai brand_segment.**

Il sistema di suggerimento tiene conto del segmento di mercato e pesa la similarità in base ad esso.
Prodotti dello stesso segmento avranno peso maggiore.

In [90]:
SEGMENT_SIMILARITY = {
    ('middle', 'middle'): 0.7,
    ('mass_market', 'mass_market'): 0.7,
    ('luxury', 'luxury'): 0.7,

    ('mass_market', 'middle'): 0.5,
    ('middle', 'mass_market'): 0.5,

    ('mass_market', 'luxury'): 0.3,
    ('luxury', 'mass_market'): 0.3,

    ('middle', 'luxury'): 0.5,
    ('luxury', 'middle'): 0.5
}


**3. Creazione dell'array di similarità 1xK.**

Per il brand non uso una cosine similarity, ma una funzione deterministica basata su regole di business che tengono conto sia dell’identità del brand sia del segmento di mercato.

Il sistema applica un hard filter sulla categoria per ridurre lo spazio di ricerca e garantire coerenza semantica, e successivamente calcola una similarità pesata basata su ingredienti e su regole di brand e segmento.



In [91]:
def brand_segment_similarity_1xK(
    query_product_id,
    df,
    category_col='macro_category',
    segment_col='brand_segment'
):
    query_product_id = str(query_product_id)

    if query_product_id not in df['code'].values:
        raise ValueError("Codice prodotto non trovato")

    query_row = df[df['code'] == query_product_id].iloc[0]
    query_category = query_row[category_col]
    query_brand = query_row['brand_name']
    query_segment = query_row[segment_col]

    # Filtro hard: stessa categoria -> ricerca tra prodotti della stessa categoria
    df_filtered = df[df[category_col] == query_category].copy()
    df_filtered = df_filtered[df_filtered['code'] != query_id]
    #filtro per evitare che restituisca lo stesso prodotto

    similarities = []

    for _, row in df_filtered.iterrows():
        if row['brand_name'] == query_brand:
            similarities.append(1.0)#se il prodotto appartiene allo stesso brand avrà similarità massima
        else:
            pair = (query_segment, row[segment_col]) #altrimenti considera il segmento di mercato
            similarities.append(SEGMENT_SIMILARITY.get(pair, 0.0))

    return np.array(similarities), df_filtered




4. Applicazione della funzione e creazione dell'aray one-time a partire dal prodotto


In [96]:
query_id = input('Inserisci il codice del prodotto: ')

brand_sim = brand_segment_similarity_1xK(query_id, df)
#non ottengo una matrice di similarità NXN, ma un array 1xK:
#cioè la similarità del prodotto query rispetto a tutti gli altri prodotti della stessa categoria



#brand_sim è una tupla: il primo elemento è l'array 1xK, il secondo elemento è il dataframe con i prodotti candidati
#brand_sim == (similarity_array, df_filtered).
#la tupla serve a sapere a quali prodotti mi riferisco
brand_sim_array, df_candidates = brand_sim



Inserisci il codice del prodotto: 5060686200113


In [97]:
brand_sim_array[:10] #vedo i primi dieci valori di similarità

array([0.7, 0.7, 0.7, 0.5, 0.5, 0.5, 0.5, 0.7, 0.5, 0.7])

In [98]:
df_candidates.head(10) #vedo i primi dieci candidati


,code,product_name,brand_name,brand_segment,macro_category
3,3560070084074,deo men marine,1 de carrefour,mass_market,Deodorants
56,4005900855848,dark forest deodorant,8x4 men,mass_market,Deodorants
58,3760346073928,deodorant bille 48h,900 care,mass_market,Deodorants
161,3700343040837,deodorant balm,acorelle,middle,Deodorants
163,3700343040899,deodorant soin,acorelle,middle,Deodorants
164,3700343040868,deodorant douceur,acorelle,middle,Deodorants
169,3700343040844,deodorant baume,acorelle,middle,Deodorants
172,3700343040400,deodorant soin,acorelle eau florale de reine des pres,mass_market,Deodorants
230,0813424021168,deodorant,acure,middle,Deodorants
254,3607342735699,get ready deo,adidas,mass_market,Deodorants


**5. Calcolo della similarità finale**

Gli ingredienti hanno peso maggiore nel determinare il prodotto più simile (0.8). I brand hanno peso minore (0.2)

In [99]:
final_similarity = (
    0.8 * ingredient_similarity +
    0.2 * brand_sim_array)


SyntaxError: incomplete input (ipython-input-3774923921.py, line 3)

**6. Costruzione dataframe per la visualizzazione del prodotto più simile**


In [ ]:
risultato = pd.DataFrame({
    'query_product_id': query_id,
    'recommended_product_id': df_filtered['code'].values,
    'final_similarity_score': final_similarity
}) #aggiungere con merge il nome prodotto se lo vogliamo per maggiore completezza

risultato = risultato.sort_values(
    by='final_similarity_score',
    ascending=False
).reset_index(drop=True)

risultato['rank'] = risultato.index + 1
